In [32]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

categoryDF=pd.read_csv("Z:/MSBooks/NLP/FinalProjectFiles/ACDFullData/aspectCategory_new.csv",delimiter='#',encoding = 'utf-8')
categoryDF.head()
train, test = train_test_split(categoryDF, test_size = 0.2)

In [33]:
train = train.reset_index(drop='True')
test = test.reset_index(drop='True')

In [34]:
trainData=[]
testData = []
trainLabels = []
testLabels = []
sentiTrainLabels=[]
sentiTestLabels=[]
for id in train.index:
    trainData.append(train.iloc[id,2])
    trainLabels.append(train.iloc[id,3])
    sentiTrainLabels.append(train.iloc[id,1])

for id in test.index:
    testData.append(test.iloc[id,2])
    testLabels.append(test.iloc[id,3])
    sentiTestLabels.append(test.iloc[id,1])

In [35]:
trainLabels = ['missing' if str(x)== 'nan' else x for x in trainLabels]
testLabels = ['missing' if str(x) == 'nan' else x for x in testLabels]

In [36]:
with open("Z:/MSBooks/NLP/FinalProjectFiles/ACDFullData/hindi_stopwords.txt") as sw:
    stopWords=[x.strip("\n") for x in sw.readlines()]


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

vectorizer = TfidfVectorizer(stop_words=stopWords,min_df=5,
                                 max_df = 0.8,
                                 sublinear_tf=True,
                                 use_idf=True)
train_vectors = vectorizer.fit_transform(trainData)
test_vectors = vectorizer.transform(testData)


classifier_rbf = svm.SVC()
classifier_rbf.fit(train_vectors, trainLabels)
prediction_rbf = classifier_rbf.predict(test_vectors)
senti_classifier_rbf=svm.SVC()
senti_classifier_rbf.fit(train_vectors,sentiTrainLabels)
senti_prediction_rbf = senti_classifier_rbf.predict(test_vectors)

In [38]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
classifier_linear = OneVsRestClassifier(SVC(kernel='linear'))
classifier_linear.fit(train_vectors, trainLabels)
prediction_linear = classifier_linear.predict(test_vectors)

senti_classifier_linear=svm.SVC(kernel='linear')
senti_classifier_linear.fit(train_vectors,sentiTrainLabels)
senti_prediction_linear = senti_classifier_linear.predict(test_vectors)


In [39]:
classifier_liblinear = svm.LinearSVC()
classifier_liblinear.fit(train_vectors, trainLabels)
prediction_liblinear = classifier_liblinear.predict(test_vectors)


senti_classifier_liblinear=svm.LinearSVC()
senti_classifier_liblinear.fit(train_vectors,sentiTrainLabels)
senti_prediction_liblinear = senti_classifier_liblinear.predict(test_vectors)

In [40]:
from sklearn.metrics import f1_score
f1_score(testLabels, prediction_linear, average='micro')  

0.53136531365313655

In [41]:
from sklearn.metrics import classification_report
target_names = set(trainLabels)
target_list = list(target_names)
print(classification_report(testLabels, prediction_linear, target_names=target_list))

                          precision    recall  f1-score   support

       hardware&software       0.00      0.00      0.00         0
            scenery&misc       0.46      0.55      0.50        84
        performance&misc       0.50      0.12      0.20         8
                    misc       0.00      0.00      0.00         3
              story&misc       0.00      0.00      0.00        22
    design&misc&hardware       0.00      0.00      0.00         1
                  budget       0.00      0.00      0.00         1
                hardware       0.00      0.00      0.00         1
              place&misc       0.00      0.00      0.00         0
                  design       0.00      0.00      0.00         3
              price&misc       0.00      0.00      0.00         1
 software&price&hardware       0.59      0.83      0.69       323
       price&design&misc       0.00      0.00      0.00        10
           hardware&misc       0.57      0.50      0.54       260
   design

In [42]:
f1_score(sentiTestLabels, senti_prediction_linear, average='micro')  

0.62638376383763839

In [43]:
from sklearn import tree
from collections import OrderedDict
mySet = set(trainLabels)
i=0
macros=OrderedDict()
reverseMap=OrderedDict()
for item in mySet:
    macros[item]=i
    reverseMap[i]=item
    i  += 1
trainMacros=[]
testMacros=[]
for item in trainLabels:
    trainMacros.append(macros[item])

    
clf = OneVsRestClassifier(tree.DecisionTreeClassifier())
clf = clf.fit(train_vectors, trainMacros)
result = clf.predict(test_vectors)    

In [44]:
resultList= result.tolist()
resultData=[]
for item in resultList:
    resultData.append(reverseMap[item])


In [45]:
f1_score(testLabels, resultData, average='micro') 

0.32472324723247231

In [46]:
from sklearn.ensemble import RandomForestClassifier
rfclf = OneVsRestClassifier(RandomForestClassifier())
rfclf.fit(train_vectors,trainMacros)
rfresult = rfclf.predict(test_vectors)
rfresultList = rfresult.tolist()
rfresultData=[]
for item in rfresultList:
    rfresultData.append(reverseMap[item])
f1_score(testLabels,rfresultData,average='micro')

0.46125461254612549